# FEATURE ENGINEERING: Military bases

In [1]:
from datetime import timedelta
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
scrubbed_data = pd.read_csv('../raw_data/scrubbed.csv', low_memory=False)

In [61]:
scrubbed_data.shape

(80332, 11)

## License conditions to use uscities.csv ('../raw_data/license.txt') 

#### Free US Cities Database: The Provider offers a free version of the US Cities Database. This Database is offered free of charge conditional on a link back to https://simplemaps.com/data/us-cities. This backlink must come from a public webpage where the Customer is using the data. If the Customer uses the data internally, the backlink must be placed on the organization's website on a page that can be easily found though links on the root domain. The link must be clearly visible to the human eye. The backlink must be placed before the Customer uses the Database in production.

In [3]:
uscities_data = pd.read_csv('../raw_data/uscities.csv'); uscities_data.shape

(28338, 17)

In [39]:
military_data = uscities_data[uscities_data['military'] == True ].copy(); military_data.shape

(85, 17)

#### Fields' description >> https://simplemaps.com/data/us-cities

In [41]:
military_data['city'] = military_data['city'].apply(lambda x: x.lower())

In [42]:
military_data.head()

,city,city_ascii,state_id,state_name,county_fips,county_name,lat,lng,population,density,source,military,incorporated,timezone,ranking,zips,id
1880,edgewood,Edgewood,MD,Maryland,24025,Harford,39.4210,-76.2968,25574,555,polygon,True,True,America/New_York,3,21009 21010 21040,1840005661
2066,fort hood,Fort Hood,TX,Texas,48027,Bell,31.1345,-97.7797,23508,579,polygon,True,True,America/Chicago,3,76544,1840073932
2458,schofield barracks,Schofield Barracks,HI,Hawaii,15003,Honolulu,21.4936,-158.0617,19499,2716,polygon,True,True,Pacific/Honolulu,2,96857 96786,1840075052
2891,fort leonard wood,Fort Leonard Wood,MO,Missouri,29169,Pulaski,37.7562,-92.1274,16026,268,polygon,True,True,America/Chicago,3,65473,1840073756
3040,fort carson,Fort Carson,CO,Colorado,8041,El Paso,38.7403,-104.7840,14951,672,polygon,True,True,America/Denver,3,80902 80913,1840028557


In [54]:
military_features = ['lat','lng','source']

In [51]:
scrubbed_features = list(scrubbed_data.columns); scrubbed_features

['datetime',
 'city',
 'state',
 'country',
 'shape',
 'duration (seconds)',
 'duration (hours/min)',
 'comments',
 'date posted',
 'latitude',
 'longitude ']

In [65]:
list_military_cities_ = list(military_data.city.unique())
list_scrubbed_cities_ = list(scrubbed_data.city.unique())

In [66]:
print(len(list_scrubbed_cities_))

19900


In [44]:
print(len(list_military_cities_)); military_data.shape

85


(85, 17)

In [62]:
result_df_ = scrubbed_data.merge(military_data,left_on='city',right_on='city', suffixes=(None,'_military'), how='left' )\
            [scrubbed_features + military_features]\
             .rename(columns={'lat':'military_lat','lng':'military_lng','source':'military_source'})

result_df_

In [63]:
print(result_df_.shape);result_df_.head()

(80332, 14)


,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude,military_lat,military_lng,military_source
0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700,45 minutes,This event took place in early fall around 194...,4/27/2004,29.8830556,-97.941111,NaN,NaN,NaN
1,10/10/1949 21:00,lackland afb,tx,NaN,light,7200,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.38421,-98.581082,29.3867,-98.6179,polygon
2,10/10/1955 17:00,chester (uk/england),NaN,gb,circle,20,20 seconds,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.2,-2.916667,NaN,NaN,NaN
3,10/10/1956 21:00,edna,tx,us,circle,20,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.9783333,-96.645833,NaN,NaN,NaN
4,10/10/1960 20:00,kaneohe,hi,us,light,900,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.4180556,-157.803611,NaN,NaN,NaN


In [64]:
result_df_.isna().sum()

datetime                    0
city                        0
state                    5797
country                  9670
shape                    1932
duration (seconds)          0
duration (hours/min)        0
comments                   15
date posted                 0
latitude                    0
longitude                   0
military_lat            80251
military_lng            80251
military_source         80251
dtype: int64

In [76]:
result_df_.fillna(value={'military_lat':0.0, 'military_lng':0.0, 'military_source':'No source'})

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude,military_lat,military_lng,military_source
0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700,45 minutes,This event took place in early fall around 194...,4/27/2004,29.8830556,-97.941111,0.0000,0.0000,No source
1,10/10/1949 21:00,lackland afb,tx,NaN,light,7200,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.38421,-98.581082,29.3867,-98.6179,polygon
2,10/10/1955 17:00,chester (uk/england),NaN,gb,circle,20,20 seconds,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.2,-2.916667,0.0000,0.0000,No source
3,10/10/1956 21:00,edna,tx,us,circle,20,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.9783333,-96.645833,0.0000,0.0000,No source
4,10/10/1960 20:00,kaneohe,hi,us,light,900,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.4180556,-157.803611,0.0000,0.0000,No source
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80327,9/9/2013 21:15,nashville,tn,us,light,600,10 minutes,Round from the distance/slowly changing colors...,9/30/2013,36.1658333,-86.784444,0.0000,0.0000,No source
80328,9/9/2013 22:00,boise,id,us,circle,1200,20 minutes,Boise&#44 ID&#44 spherical&#44 20 min&#44 10 r...,9/30/2013,43.6136111,-116.202500,0.0000,0.0000,No source
80329,9/9/2013 22:00,napa,ca,us,other,1200,hour,Napa UFO&#44,9/30/2013,38.2972222,-122.284444,0.0000,0.0000,No source
80330,9/9/2013 22:20,vienna,va,us,circle,5,5 seconds,Saw a five gold lit cicular craft moving fastl...,9/30/2013,38.9011111,-77.265556,0.0000,0.0000,No source


In [77]:
result_df_.to_csv('../raw_data/scrubbed_with_military_bases.csv',header=True,index=False)